## 4.2 Data Quality Checks

Data quality checks includes

Data schema of every dimensional table matches data model
No empty table after running ETL data pipeline

In [ ]:
import os
import configparser
from pathlib import Path
from pyspark.sql import SparkSession

: 

In [ ]:
config = configparser.ConfigParser()
config.read('../config/.env', encoding='utf-8-sig')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']
SOURCE_S3_BUCKET = config['S3']['SOURCE_S3_BUCKET']
DEST_S3_BUCKET = config['S3']['DEST_S3_BUCKET']

In [ ]:
spark = SparkSession.builder\
                    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0")\
                    .enableHiveSupport().getOrCreate()

### 1. Data schema of every dimensional table matches data model

In [ ]:
bucket = Path(DEST_S3_BUCKET)

In [ ]:
for dir in bucket.iterdir():
    if dir.is_dir():
        path = str(dir)
        df = spark.read.parquet(path)
        print("Table: " + path.split('/')[-1])
        schema = df.printSchema()

### 2. Data quality on table check

In [ ]:
for file_dir in bucket.iterdir():
    path = str(file_dir)
    df = spark.read.parquet(path)
    records = df.count()
    table = path.split('/')[-1]
    print(f"Data quality on table {table} check passed with {records} records")